In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 0) Cargar datos
df = pd.read_csv("./dataset_definitivo/dataset_definitivo.csv")

# 1) Limpieza mínima para evitar los problemas con el target 

df = df.dropna(subset=["NUM_INFRACCIONES", "CUANTIA"]).copy()

# 2) Split ANTES de crear el target 
X_full = df.copy()

X_train_raw, X_test_raw = train_test_split(
    X_full,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# 3) Calcular umbrales SOLO en TRAIN
p80_infracciones = X_train_raw["NUM_INFRACCIONES"].quantile(0.80)
p80_cuantia      = X_train_raw["CUANTIA"].quantile(0.80)

# 4) Crear el target en TRAIN y TEST usando UMBRALES DE TRAIN (no del test)
def crear_target_alto_riesgo(df_in: pd.DataFrame,
                             thr_infr: float,
                             thr_cuant: float) -> pd.Series:
    return np.where(
        (df_in["NUM_INFRACCIONES"] >= thr_infr) | (df_in["CUANTIA"] >= thr_cuant),
        1, 0
    )

y_train = crear_target_alto_riesgo(X_train_raw, p80_infracciones, p80_cuantia)
y_test  = crear_target_alto_riesgo(X_test_raw,  p80_infracciones, p80_cuantia)

# 5) Construir X eliminando lo que define el target (las dos de antes vaya)
cols_leak = ["NUM_INFRACCIONES", "CUANTIA"]  
X_train = X_train_raw.drop(columns=cols_leak).copy()
X_test  = X_test_raw.drop(columns=cols_leak).copy()

# 6) Checks rápidos para ver que sale mas o menos 
print("Umbrales (train):", p80_infracciones, p80_cuantia)
print("Train size:", X_train.shape, " Test size:", X_test.shape)
print("Distribución y_train:", pd.Series(y_train).value_counts(normalize=True))
print("Distribución y_test :", pd.Series(y_test).value_counts(normalize=True))

Umbrales (train): 4.0 500.0
Train size: (252187, 10)  Test size: (63047, 10)
Distribución y_train: 0    0.560195
1    0.439805
Name: proportion, dtype: float64
Distribución y_test : 0    0.563643
1    0.436357
Name: proportion, dtype: float64


## **En resumen**:

### Definición del target "ALTO_RIESGO"

Los umbrales utilizados corresponden al **percentil 80 (p80)** calculado únicamente sobre el conjunto de entrenamiento (`train`):

- `p80_infracciones`: valor a partir del cual un conductor está en el 20% superior en número de infracciones.
- `p80_cuantia`: valor a partir del cual un conductor está en el 20% superior en cuantía económica de sanciones.

#### Regla de asignación del target

Se asigna:

- **1 (Alto Riesgo)** → Si el conductor cumple:
  - `NUM_INFRACCIONES ≥ p80_infracciones`  
  **o**
  - `CUANTIA ≥ p80_cuantia`

- **0 (Riesgo no alto)** → Si no cumple ninguna de las dos condiciones anteriores.

Es decir, se considera incidencia grave cuando el conductor se sitúa en el 20% superior en al menos una de las dos variables.


### Distribución de clases tras el split

**Train**
- Clase 0 → 56.02%
- Clase 1 → 43.98%

**Test**
- Clase 0 → 56.36%
- Clase 1 → 43.64%

La proporción entre train y test es muy similar, lo que indica que el split mantiene coherencia en la distribución del target.

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 0) Cargar datos
df = pd.read_csv("./dataset_definitivo/dataset_definitivo.csv")

# 1) Limpieza mínima
needed = ["NUM_INFRACCIONES", "CUANTIA", "PUNTOS"]
df = df.dropna(subset=needed).copy()

# 2) Split *antes* de crear el target / umbrales
X_full = df.copy()
X_train_raw, X_test_raw = train_test_split(
    X_full,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# 3) Crear severity score basado en percentiles (rank pct) calculados SOLO con TRAIN
# Básicamente para tener en cuenta por igual las 3 variables ya que trabajan en rangos diferentes
# Como cuando escalabamos en regresión

def add_severity_score(df_in: pd.DataFrame, ref_train: pd.DataFrame) -> pd.DataFrame:
    out = df_in.copy()

    def pct_rank_from_train(x: pd.Series, train_series: pd.Series) -> pd.Series:
       
        train_vals = np.sort(train_series.to_numpy())
        return pd.Series(np.searchsorted(train_vals, x.to_numpy(), side="right") / len(train_vals), index=x.index)

    out["pr_infr"]  = pct_rank_from_train(out["NUM_INFRACCIONES"], ref_train["NUM_INFRACCIONES"])
    out["pr_cuant"] = pct_rank_from_train(out["CUANTIA"],          ref_train["CUANTIA"])
    out["pr_pts"]   = pct_rank_from_train(out["PUNTOS"],         ref_train["PUNTOS"])

    # pesos (me los ha dado chat gpt)
    w_pts, w_infr, w_cuant = 0.40, 0.35, 0.25
    out["severity_score"] = (
        w_pts  * out["pr_pts"] +
        w_infr * out["pr_infr"] +
        w_cuant* out["pr_cuant"]
    )
    return out

train_scored = add_severity_score(X_train_raw, X_train_raw)
test_scored  = add_severity_score(X_test_raw,  X_train_raw)

# 4) Cortes en 5 categorías (definidos SOLO con TRAIN y aplicados a TEST)

cuts = train_scored["severity_score"].quantile([0.60, 0.75, 0.85, 0.95]).to_dict()
c60, c75, c85, c95 = cuts[0.60], cuts[0.75], cuts[0.85], cuts[0.95]

labels = ["muy_baja", "baja", "media", "alta", "muy_alta"]

def to_5cats(score: pd.Series) -> pd.Categorical:
    return pd.cut(
        score,
        bins=[-np.inf, c60, c75, c85, c95, np.inf],
        labels=labels,
        include_lowest=True
    )

y_train = to_5cats(train_scored["severity_score"])
y_test  = to_5cats(test_scored["severity_score"])

# 5) Construir X eliminando las columnas que definen el target para evitar leakage (las tres de antes otra vez vaya)
cols_leak = ["NUM_INFRACCIONES", "CUANTIA", "PUNTOS"]
X_train = X_train_raw.drop(columns=cols_leak).copy()
X_test  = X_test_raw.drop(columns=cols_leak).copy()

# 6) Salidas finales en esas 5 cat
print("Cortes severity_score (train):", {"p60": c60, "p75": c75, "p85": c85, "p95": c95})
print("Train size:", X_train.shape, " Test size:", X_test.shape)

print("\nDistribución y_train (5 categorías):")
print(pd.Series(y_train).value_counts(normalize=True).reindex(labels).fillna(0))

print("\nDistribución y_test (5 categorías):")
print(pd.Series(y_test).value_counts(normalize=True).reindex(labels).fillna(0))

Cortes severity_score (train): {'p60': 0.700863248303838, 'p75': 0.7583725568724795, 'p85': 0.8202433115109027, 'p95': 0.9023331892603504}
Train size: (252187, 9)  Test size: (63047, 9)

Distribución y_train (5 categorías):
severity_score
muy_baja    0.614584
baja        0.141125
media       0.094688
alta        0.119741
muy_alta    0.029863
Name: proportion, dtype: float64

Distribución y_test (5 categorías):
severity_score
muy_baja    0.619522
baja        0.140689
media       0.092169
alta        0.117563
muy_alta    0.030057
Name: proportion, dtype: float64


## Construcción del `severity_score`

El `severity_score` se construye como una combinación ponderada de tres variables:

- `CUANTIA`
- `PUNTOS`
- `NUM_INFRACCIONES`

Primero, cada variable se normaliza en rango 0–1 (para que sean comparables y adimensionales).

Después, se aplica una mini-ecuación con pesos:

severity_score = w1·CUANTIA_norm + w2·PUNTOS_norm + w3·NUM_INFRACCIONES_norm

Donde:

- w1, w2, w3 = pesos asignados según la importancia relativa que queremos dar a cada variable.
- El resultado es un coeficiente continuo entre 0 y 1.


## Conversión del score en categorías

Una vez obtenido el `severity_score`, se clasifica usando percentiles del conjunto de entrenamiento:

- ≤ p60 → muy_baja  
- p60–p75 → baja  
- p75–p85 → media  
- p85–p95 → alta  
- `>` p95 → muy_alta  

## Cómo modificar el modelo

Este enfoque es flexible:

1. **Cambiar la importancia relativa** → Ajustando los pesos (w1, w2, w3).
2. **Cambiar la distribución de categorías** → Modificando los percentiles de corte.

De esta forma, podemos adaptar:
- Qué variables influyen más en la severidad.
- Qué proporción del dataset consideramos “riesgo alto” o “riesgo extremo”.

Esto permite ajustar el modelo si la división inicial no refleja bien la realidad que queremos representar, eso ya lo veremos pero vamos que de momento lo veo bastante bien.